In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor, XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
# Настройка визуализаций
%matplotlib inline
df = pd.read_excel('data/data.xlsx', index_col='Unnamed: 0')
# т. к. пустых значений не много, а заполнять их не представляется возможным - удаляем их
df.dropna(how='any', inplace=True)

# Классификация: превышает ли значение IC50 медианное значение выборки

In [2]:
# Создадим столбец, который будет содержать 2 поля: 1 если значение превышает медиану и 
# 0 в обратном случае
df['Class_1'] = [np.nan]*len(df)
df.loc[df['IC50, mM']>=df['IC50, mM'].median(), 'Class_1'] = 1
df.loc[df['IC50, mM']<df['IC50, mM'].median(), 'Class_1'] = 0
X = df.drop(['IC50, mM', 'CC50, mM', 'SI', 'Class_1'], axis=1)
y = df['Class_1']

In [3]:
# Отбор важных признаков
selector_forest = SelectFromModel(
    RandomForestClassifier(n_estimators=100),
    threshold="median"
)
selector_forest.fit(X, y)

selected_features = list(X.columns[selector_forest.get_support()])
selected_features = list(set(selected_features))

print(f"Всего отобрано признаков: {len(selected_features)}")

Всего отобрано признаков: 105


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X[selected_features], y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42))
])

param_grid = {
    'clf__n_estimators': [100, 200, 300],
    'clf__max_depth': [5, 10, 15, None],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['sqrt', 'log2', None]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nМодель классификации случайного леса:")
print("Лучшие параметры:", grid_search.best_params_)
print(f"Основные метрики: {classification_report(y_test, y_pred)}")
print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")


Модель классификации случайного леса:
Лучшие параметры: {'clf__max_depth': 5, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 5, 'clf__n_estimators': 100}
Основные метрики:               precision    recall  f1-score   support

         0.0       0.74      0.67      0.70       100
         1.0       0.70      0.76      0.73       100

    accuracy                           0.71       200
   macro avg       0.72      0.72      0.71       200
weighted avg       0.72      0.71      0.71       200

Confusion Matrix: [[67 33]
 [24 76]]


In [5]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 7],
    'learning_rate': [0.05, 0.1, 0.15],
    'gamma': [0, 0.1],
    'reg_alpha': [0, 0.1],
    'subsample': [0.8, 0.9], 
    'colsample_bytree': [0.8, 0.9]
}

xgb_classifier = XGBClassifier(
        objective='binary:logistic', # т. к. только 2 класса
        random_state=42
    )

grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    n_jobs=-1,
)

# Запуск поиска
grid_search.fit(X_train, y_train)

y_pred = grid_search.predict(X_test)

print("\nМодель классификации XGB:")
print("Лучшие параметры:", grid_search.best_params_)
print(f"Основные метрики: {classification_report(y_test, y_pred)}")
print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")

C:\Users\raven\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,



Модель классификации XGB:
Лучшие параметры: {'colsample_bytree': 0.9, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'reg_alpha': 0.1, 'subsample': 0.8}
Основные метрики:               precision    recall  f1-score   support

         0.0       0.75      0.68      0.71       100
         1.0       0.71      0.77      0.74       100

    accuracy                           0.72       200
   macro avg       0.73      0.73      0.72       200
weighted avg       0.73      0.72      0.72       200

Confusion Matrix: [[68 32]
 [23 77]]


Проанализировав результаты двух моделей можно сделать вывод, что использование XGBClassifier улучшило все показатели модели, что говорит о том, что предпочтительно использовать именно его.